In [1]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from epi_ml.core.metadata import Metadata
from epi_ml.utils.general_utility import get_valid_filename

## Setup

In [2]:
ASSAY = "assay_epiclass"
SEX = "harmonized_donor_sex"
TRACK = "track_type"

In [3]:
epilap_project = Path.home() / "Projects/epilap"

metadata = (
    epilap_project
    / "input"
    / "metadata"
    / "dfreeze-v2"
    / "hg38_2023-epiatlas-dfreeze_v2.1_w_encode_noncore_2.json"
)

logs = epilap_project / "output/logs/epiatlas-dfreeze-v2.1"
chrY_coverage_file = logs / "chrY_coverage_results" / "chrXY_coverage_all.csv"

In [4]:
meta = Metadata(metadata)

In [5]:
df_chrY = pd.read_csv(chrY_coverage_file, index_col=0, header=0)

In [6]:
print(df_chrY.columns)

Index(['chrY', 'chrX', 'chrY/chrX'], dtype='object')


## Merge metadata and chrY coverage info

In [53]:
meta_df = pd.DataFrame.from_records(list(meta.datasets)).set_index("md5sum")

In [54]:
meta_df[ASSAY].value_counts()

h3k27ac          4689
non-core         3599
h3k4me1          2889
h3k4me3          2397
rna_seq          2309
h3k36me3         2085
h3k27me3         2025
h3k9me3          1926
wgbs-standard     884
input             777
mrna_seq          681
CTCF              468
wgbs-pbat         260
Name: assay_epiclass, dtype: int64

In [9]:
# remove non-core tracks
# meta_df = meta_df[~ meta_df[ASSAY].str.contains("non-core|CTCF")]

In [55]:
merged_df = df_chrY.merge(meta_df, left_index=True, right_index=True)

In [57]:
print(meta_df.shape, df_chrY.shape, merged_df.shape)

(24989, 169) (21789, 3) (20922, 172)


In [56]:
# Check the merge
assert (merged_df.shape[0] == meta_df.shape[0]) and (
    df_chrY.shape[1] + meta_df.shape[1] == merged_df.shape[1]
), "not right shape"

AssertionError: not right shape

In [13]:
# for item in set(meta_df.index.values) - set(df_chrY.index.values):
#     print(item)

## Compute chrY zscore coverage by assay and sex

In [14]:
# groupby_columns = [ASSAY, SEX, TRACK]
# groupby_columns = [ASSAY, SEX]
groupby_columns = [ASSAY]

chrY_dists = merged_df.groupby(groupby_columns).agg({"chrY": ["mean", "std", "count"]})
display(chrY_dists)
# display(chrY_dists.index)

chrY                
                    mean       std count
assay_epiclass                          
h3k27ac         0.102679  0.144223  4689
h3k27me3        0.103248  0.208041  2025
h3k36me3        0.091796  0.165159  2085
h3k4me1         0.081342  0.149748  2889
h3k4me3         0.084939  0.115496  2397
h3k9me3         0.166447  0.307748  1926
input           0.208890  0.314801   777
mrna_seq        0.049679  0.076258   681
rna_seq         0.042931  0.047574  2309
wgbs-pbat       7.444794  6.063828   260
wgbs-standard   4.475153  4.275084   884

In [15]:
partial_name = "assay"

In [16]:
# chrY_dists.to_csv(logs / "chrY_coverage_results" / f"chrY_coverage_distributions_{partial_name}.csv")

In [17]:
score_name = f"expected_{partial_name}_chrY_z-score"

In [18]:
new_data = []
for row in chrY_dists.iterrows():
    index, vals = row
    # assay, sex, track = index # type: ignore
    # assay, sex = index # type: ignore
    assay = index  # type: ignore

    mean = vals["chrY"]["mean"]
    std = vals["chrY"]["std"]
    count = vals["chrY"]["count"]

    # partial_df = merged_df[(merged_df[ASSAY] == assay) & (merged_df[SEX] == sex) & (merged_df[TRACK] == track)]
    # partial_df = merged_df[(merged_df[ASSAY] == assay) & (merged_df[SEX] == sex)]
    partial_df = merged_df[(merged_df[ASSAY] == assay)]
    partial_df = partial_df[["chrY"]].copy()

    partial_df.loc[:, score_name] = (partial_df["chrY"] - mean) / std
    partial_df.loc[:, f"count_{score_name}"] = count

    new_data.append(partial_df)

In [19]:
# print(new_data)

In [20]:
all_zscores_df = pd.concat(new_data)

In [21]:
final_df = df_chrY.merge(
    all_zscores_df, how="left", left_index=True, right_index=True, suffixes=("", "_DROP")
).sort_index()
final_df = final_df.drop(columns=["chrY_DROP"])

In [22]:
# final_df.to_csv(logs / "chrY_coverage_results" / f"chrY_coverage_zscores_{partial_name}.csv")

## Map predictions to chrY z-scores

In [23]:
sex_pred_dir = Path.home() / "downloads" / "temp"
sex_pred_file = (
    sex_pred_dir / "sex3_oversample_full-10fold-validation_prediction_augmented-all.csv"
)

full_chrY_df = pd.read_csv(
    logs / "chrY_coverage_results" / f"chrY_coverage_zscores.csv", index_col=0, header=0
)

In [24]:
pred_df = pd.read_csv(sex_pred_file, index_col=0, header=0)
print(pred_df.shape)

(18299, 87)


In [58]:
merged_pred_df = pred_df.merge(
    full_chrY_df, how="inner", left_index=True, right_index=True, suffixes=("", "_DROP")
).sort_index()
merged_pred_df = merged_pred_df.drop(
    columns=[column for column in pred_df.columns if column.endswith("_DROP")]
)
print(merged_pred_df.shape)

(18299, 96)


In [26]:
raise UserWarning("stop here")

UserWarning: stop here

## chrY z-scores confusion matrix for sex3 preds

In [ ]:
COLORS_DICT = {"female": "red", "male": "blue", "mixed": "purple"}

In [ ]:
# merged_pred_df.to_csv(sex_pred_dir / "sex3_oversample_full-10fold-validation_prediction_augmented-all_chrY_zscores.csv")

In [ ]:
print([col for col in merged_pred_df.columns if "z-score" in col])

In [ ]:
# merged_pred_df = merged_pred_df[
#     ~merged_pred_df[ASSAY].str.contains(pat="wgb", case=False)
# ]

### graphs per assay

In [ ]:
coverage_label = "expected_assay_chrY_z-score"
count_label = f"count_{coverage_label}"

classes = merged_pred_df["True class"].unique()
assays = merged_pred_df[ASSAY].unique()

matrix_logdir = chrY_coverage_file = (
    logs
    / "chrY_coverage_results"
    / "10fold_valid"
    / "z-score"
    / "per_assay"
    / "w-unknown"
    / "per_assay_graph"
)

for assay_label in assays:
    assay_df = merged_pred_df[merged_pred_df[ASSAY] == assay_label]

    # confusion matrix for chrY z-score
    for threshold in [0, 0.7, 0.9]:
        row = 1
        col = 1
        fig = make_subplots(
            rows=3,
            cols=3,
            shared_yaxes=True,
            x_title="Predicted class (nb of predictions)",
            y_title="z-score vs expected assay",
            row_titles=list(classes),
            column_titles=list(classes),
            vertical_spacing=0.08,
            horizontal_spacing=0.01,
        )
        threshold_df = assay_df[assay_df["Max pred"] >= threshold]

        for label in classes:
            df_label = threshold_df[threshold_df["True class"] == label]

            # Iterate over each target and add a violin plot for it
            for target in classes:
                sub_df = df_label[df_label["Predicted class"] == target]

                if sub_df.shape[0] == 0:
                    y_values = [
                        threshold_df[coverage_label].mean()
                    ]  # Minimal synthetic data
                    sample_count = 0
                    hovertext = ["PLACEHOLDER - NO DATA"]
                else:
                    y_values = sub_df[coverage_label]
                    hovertext = [
                        f"{md5sum, assay}:(z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f})"
                        for md5sum, pred, z_score, count, assay in zip(
                            sub_df.index,
                            sub_df["Max pred"],
                            sub_df[coverage_label],
                            sub_df[count_label],
                            sub_df[ASSAY],
                        )
                    ]

                fig.add_trace(
                    go.Violin(
                        y=y_values,
                        name=f"{target} ({sub_df.shape[0]})",
                        box_visible=True,
                        meanline_visible=True,
                        points="all",
                        text=hovertext,
                        line_color=COLORS_DICT[target],
                        hovertemplate="%{text}",
                    ),
                    row=row,
                    col=col,
                )

                # Move to the next subplot position
                col += 1
                if col > 3:
                    col = 1
                    row += 1

        # Update global layout and traces
        fig.update_traces(marker=dict(size=1))
        fig.update_yaxes(
            range=[
                min(assay_df[coverage_label]) - 0.01,
                max(assay_df[coverage_label]) + 0.01,
            ]
        )

        # Directly using annotations param does not work with make_subplots
        existing_annotations = fig.layout.annotations
        new_annotation = dict(
            x=1.01,  # Position on the x-axis
            y=0.5,  # Position on the y-axis
            showarrow=False,  # Do not show arrow
            text="Reference class",  # The text you want to display
            xref="paper",  # 'x' coordinate is set in relative coordinates
            yref="paper",  # 'y' coordinate is set in relative coordinates
            xanchor="left",  # Text starts from the left of the x-coordinate
            yanchor="middle",  # Middle aligned vertically
            font=dict(size=16),
            textangle=90,
        )
        updated_annotations = list(existing_annotations) + [new_annotation]

        title = f"z-score(mean chrY coverage per file):{assay_label} (pred>{threshold})"

        fig.update_layout(
            title_text=f"{title} (n={threshold_df.shape[0]})",
            showlegend=False,
            annotations=updated_annotations,
        )

        # fig.show()

        title = get_valid_filename(title).replace("_br_", "_")
        html_file = matrix_logdir / f"{title}.html"
        png_file = matrix_logdir / f"{title}.png"
        if not png_file.exists():
            fig.write_image(png_file, scale=2)
        if not html_file.exists():
            fig.write_html(html_file)

### Graphs for all assays mixed

In [ ]:
coverage_label = "expected_assay_chrY_z-score"
count_label = f"count_{coverage_label}"

classes = merged_pred_df["True class"].unique()

matrix_logdir = chrY_coverage_file = (
    logs
    / "chrY_coverage_results"
    / "10fold_valid"
    / "z-score"
    / "per_assay"
    / "w-unknown"
)


# confusion matrix for chrY z-score
for threshold in [0, 0.7, 0.9]:
    row = 1
    col = 1
    fig = make_subplots(
        rows=3,
        cols=3,
        shared_yaxes=True,
        x_title="Predicted class (nb of predictions)",
        y_title="z-score for expected assay",
        row_titles=list(classes),
        column_titles=list(classes),
        vertical_spacing=0.08,
        horizontal_spacing=0.01,
    )
    threshold_df = merged_pred_df[merged_pred_df["Max pred"] >= threshold]

    title = (
        f"z-score(mean chrY coverage per file) - (pred>{threshold})<br>w fc/pval, no wgbs"
    )

    filename = get_valid_filename(title).replace("_br_", "_")
    html_file = matrix_logdir / f"{filename}.html"
    png_file = matrix_logdir / f"{filename}.png"
    if png_file.exists() or html_file.exists():
        continue

    for label in classes:
        df_label = threshold_df[threshold_df["True class"] == label]

        # Iterate over each target and add a violin plot for it
        for target in classes:
            sub_df = df_label[df_label["Predicted class"] == target]

            if sub_df.shape[0] == 0:
                y_values = [threshold_df[coverage_label].mean()]  # Minimal synthetic data
                sample_count = 0
                hovertext = ["PLACEHOLDER - NO DATA"]
            else:
                y_values = sub_df[coverage_label]
                hovertext = [
                    f"{md5sum, assay}:(z-score={z_score:.3f} (n={int(count)}), pred={pred:.3f})"
                    for md5sum, pred, z_score, count, assay in zip(
                        sub_df.index,
                        sub_df["Max pred"],
                        sub_df[coverage_label],
                        sub_df[count_label],
                        sub_df[ASSAY],
                    )
                ]

            fig.add_trace(
                go.Violin(
                    y=y_values,
                    name=f"{target} ({sub_df.shape[0]})",
                    box_visible=True,
                    meanline_visible=True,
                    points="all",
                    text=hovertext,
                    line_color=COLORS_DICT[target],
                    hovertemplate="%{text}",
                ),
                row=row,
                col=col,
            )

            # Move to the next subplot position
            col += 1
            if col > 3:
                col = 1
                row += 1

    # Update global layout and traces
    fig.update_traces(marker=dict(size=1))
    fig.update_yaxes(
        range=[
            min(merged_pred_df[coverage_label]) - 0.01,
            max(merged_pred_df[coverage_label]) + 0.01,
        ]
    )

    # Directly using annotations param does not work with make_subplots
    existing_annotations = fig.layout.annotations  # type: ignore
    new_annotation = dict(
        x=1.01,  # Position on the x-axis
        y=0.5,  # Position on the y-axis
        showarrow=False,  # Do not show arrow
        text="Reference class",  # The text you want to display
        xref="paper",  # 'x' coordinate is set in relative coordinates
        yref="paper",  # 'y' coordinate is set in relative coordinates
        xanchor="left",  # Text starts from the left of the x-coordinate
        yanchor="middle",  # Middle aligned vertically
        font=dict(size=16),
        textangle=90,
    )
    updated_annotations = list(existing_annotations) + [new_annotation]

    fig.update_layout(
        title_text=f"{title} (n={threshold_df.shape[0]})",
        showlegend=False,
        annotations=updated_annotations,
    )

    fig.show()

    fig.write_image(png_file, scale=2)
    fig.write_html(html_file)

## Merge with global results file

In [47]:
target_file_dir = logs / "merged_results"
target_file = target_file_dir / "merged_pred_results_all_2.1_chrY.csv"

target_df = pd.read_csv(target_file, index_col=0, header=0)

/tmp/ipykernel_336231/4197668515.py:4: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,144,145,146,147,148,149,151,160,161,162,164,186,187,188,190,199,200,201,203,212,213,214,216,225,226,227,229,238,239,240,242,251,252,253,255,264,265,266,268,277,278,279,281,290,291,292,294,303,304,305,307,316,317,318,320,331,342,343,344,346,355,356,357,359) have mixed types. Specify dtype option on import or set low_memory=False.
  target_df = pd.read_csv(target_file, index_col=0, header=0)


In [48]:
updated_target_df = target_df.merge(
    full_chrY_df, how="left", left_index=True, right_index=True, suffixes=("", "_DROP")
).sort_index()

In [49]:
same_cols = []
for column in updated_target_df.columns:
    if column.endswith("_DROP"):
        same_cols.append(column[:-5])

In [50]:
for column in same_cols:
    assert np.isclose(
        0, (updated_target_df[column] - updated_target_df[f"{column}_DROP"]).sum()
    ), f"{column} not equal"

In [51]:
updated_target_df = updated_target_df.drop(
    columns=[column for column in updated_target_df.columns if column.endswith("_DROP")]
)

In [52]:
updated_target_df.to_csv(target_file_dir / "merged_pred_results_all_2.1_chrY_zscores.csv")